In [3]:
import pandas as pd
import numpy as np
import os
import glob

In [ ]:
# get all csv file paths from data folder
csv_files = glob.glob("data/CGMacros-*.csv")  

# list storing data frames per participant
data_list = []

for file in csv_files:
    df = pd.read_csv(file)
    df["participant"] = int(file.split("-")[-1].split(".")[0]) 
    df.drop(columns=["Image path"], errors="ignore", inplace=True)
    temp_df = df.copy()
    data_list.append(temp_df)  

merged_df = pd.concat(data_list, ignore_index=True)

In [13]:
merged_df.to_csv("all_participants.csv", index=False)

In [14]:
merged_df

,Unnamed: 0,Timestamp,Libre GL,Dexcom GL,HR,Calories (Activity),METs,Meal Type,Calories,Carbs,Protein,Fat,Fiber,Amount Consumed,participant,Amount Consumed,Steps,RecordIndex,Intensity,Sugar
0,0.0,2020-05-01 10:30:00,84.000000,NaN,56.0,1.0484,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN
1,1.0,2020-05-01 10:31:00,84.133333,NaN,56.0,1.0484,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN
2,2.0,2020-05-01 10:32:00,84.266667,NaN,57.0,1.0484,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN
3,3.0,2020-05-01 10:33:00,84.400000,NaN,54.0,1.0484,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN
4,4.0,2020-05-01 10:34:00,84.533333,NaN,55.0,1.0484,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
687575,15310.0,2025-05-22 00:03:00,262.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,49,NaN,NaN,NaN,NaN,NaN
687576,15311.0,2025-05-22 00:04:00,261.600000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,49,NaN,NaN,NaN,NaN,NaN
687577,15312.0,2025-05-22 00:05:00,261.200000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,49,NaN,NaN,NaN,NaN,NaN
687578,15313.0,2025-05-22 00:06:00,260.800000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,49,NaN,NaN,NaN,NaN,NaN


In [ ]:
'CGMacros-001.csv'.split("-")[-1].split(".")[0] 

'001'

In [11]:
int('CGMacros-001.csv'.split("-")[-1].split(".")[0])

1

In [ ]:

# Path to the folder containing all participant CSVs
data_folder = "/data"
bio_path = "bio.csv"  # Path to bio.csv

# Get all CSV file paths
csv_files = glob.glob(os.path.join(data_folder, "*.csv"))

# Create an empty list to store DataFrames
dfs = []

# Loop through all CSVs
for file in csv_files:
    participant_id = int(os.path.basename(file).split('.')[0].replace("cgm", ""))  # Extract participant number
    df = pd.read_csv(file)
    
    # Drop the "Image path" column
    df.drop(columns=["Image path"], inplace=True, errors="ignore")
    
    # Add participant ID
    df["Participant ID"] = participant_id
    
    dfs.append(df)

# Combine all participant data into one DataFrame
full_data = pd.concat(dfs, ignore_index=True)

# Load bio.csv
bio_df = pd.read_csv(bio_path)

# Keep only necessary columns from bio.csv
bio_df = bio_df[["subject", "A1c PDL (Lab)"]].rename(columns={"subject": "Participant ID", "A1c PDL (Lab)": "HbA1c"})

# Merge full data with bio.csv
merged_data = full_data.merge(bio_df, on="Participant ID", how="left")

# Classify participants into Diabetes Status groups
def classify_diabetes(hba1c):
    if hba1c < 5.7:
        return "No Diabetes"
    elif 5.7 <= hba1c <= 6.4:
        return "Pre-Diabetes"
    else:
        return "Type 2 Diabetes"

merged_data["Diabetes Status"] = merged_data["HbA1c"].apply(classify_diabetes)

# Split into separate DataFrames
no_diabetes_df = merged_data[merged_data["Diabetes Status"] == "No Diabetes"]
pre_diabetes_df = merged_data[merged_data["Diabetes Status"] == "Pre-Diabetes"]
t2d_df = merged_data[merged_data["Diabetes Status"] == "Type 2 Diabetes"]

# Save to CSV if needed
no_diabetes_df.to_csv("no_diabetes_data.csv", index=False)
pre_diabetes_df.to_csv("pre_diabetes_data.csv", index=False)
t2d_df.to_csv("type2_diabetes_data.csv", index=False)

print("Data processing complete!")
